Tensorflow Object detection Using Efficientdet_d7_coco_tpu-32

In [1]:
#import neccessary libraries

import tensorflow as tf 
import cv2 as cv 
import numpy as np

In [2]:
#load tfod model
model_path = "efficientdet_d2_coco17_tpu-32/saved_model"
model = tf.keras.models.load_model(model_path)

C:\Users\XPRESS\anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\layers\core\lambda_layer.py:297: UserWarning: google3.third_party.tensorflow.python.ops.nn_impl is not loaded, but a Lambda layer uses it. It may cause errors.
  function = cls._parse_function_from_config(config, custom_objects,
C:\Users\XPRESS\anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\layers\core\lambda_layer.py:297: UserWarning: google3.third_party.tensorflow_models.object_detection.utils.bifpn_utils is not loaded, but a Lambda layer uses it. It may cause errors.
  function = cls._parse_function_from_config(config, custom_objects,


In [3]:
#read the labels of the detection 

with open('coco.names','r') as f:
    names = f.read().splitlines()

In [4]:
img =cv.imread(r"crosswalk-featured.jpg")
tensorimg = cv.cvtColor(img,cv.COLOR_BGR2RGB)  #convert BGR image to RGB image
tensorimg = tf.convert_to_tensor(tensorimg.copy(),dtype=tf.uint8) #convert image to tensorflw tensor
tensorimg = tensorimg[tf.newaxis,...] #expand dimensions 

detection = model(tensorimg)
detection_boxes = detection["detection_boxes"][0].numpy()
detection_classes = detection["detection_classes"][0].numpy().astype(np.int32)
detection_scores = detection["detection_scores"][0].numpy()



random_color = np.random.uniform(low=0,high=255,size=(80,3))

bboxIdx = tf.image.non_max_suppression(detection_boxes,detection_scores,max_output_size=50,iou_threshold=0.5,score_threshold=0.5)


#function to create bounding box

def create_bbox(image):
    imH,imW,imC = image.shape
    for i in bboxIdx:
        bbox = tuple(detection_boxes[i].tolist())
        classname = detection_classes[i]
        classname = names[classname-1]
        ymin,xmin,ymax,xmax =bbox
        ymin,xmin,ymax,xmax = (ymin*imH,xmin*imW,ymax*imH,xmax*imW)
        ymin = int(ymin)
        xmin = int(xmin)
        ymax = int(ymax)
        xmax = int(xmax)
        
        # print(ymin,ymax,xmin,xmax)
        center1 = int((xmin+xmax)/2)
        center2 = int((ymin+ymax)/2)
        center3 = np.random.randint(xmax/2,xmax)
        center4 = np.random.randint(ymin,ymax)
        center5 = np.random.randint(ymin,ymax)
    
        print(center3,center4)
        if (center3-center4) < 200:
            center3 += 100
            center4 += 100
        else:
            pass
        
        color =random_color[i]
        print(color)
    
        cv.rectangle(image,(xmin,ymin),(xmax,ymax),(color),1)
        cv.line(image,(xmin,ymin),(xmin+20,ymin),(color),3)
        cv.line(image,(xmin,ymin),(xmin,ymin+20),(color),3)
        cv.line(image,(xmax-20,ymin),(xmax,ymin),(color),3)
        cv.line(image,(xmax,ymin),(xmax,ymin+20),(color),3)
        cv.line(image,(xmax,ymax),(xmax-20,ymax),(color),3)
        cv.line(image,(xmax,ymax-20),(xmax,ymax),(color),3)
        cv.line(image,(xmin+20,ymax),(xmin,ymax),(color),3)
        cv.line(image,(xmin,ymax-20),(xmin,ymax),(color),3)
        cv.putText(image,(classname.upper()),(xmin,ymin-5),cv.FONT_HERSHEY_PLAIN,1.0,(color),2)


    return image





image = create_bbox(img)
cv.imshow("img",image)
cv.imwrite("testedimage.jpg",image)


cv.waitKey(0)
cv.destroyAllWindows()


497 410
[228.87933314 236.4999965  124.85913388]
973 196
[251.70585696 174.78571049  91.77112615]
405 276
[183.43198739  47.75261664 158.27050987]
176 478
[106.50791224  77.01792756  29.91212854]
992 248
[54.9536257  99.38343364 54.19087913]
609 258
[213.5113928  249.98298518 155.47401067]
272 265
[172.15861131 136.64595591 150.92575178]
184 262
[161.0933345  119.70584127 227.09603176]
825 251
[206.23108502   1.62212549 178.29777162]
309 463
[  7.95900561 227.65643781 188.15348327]
830 417
[ 88.58050786  36.46899279 233.03215589]
829 266
[181.30643023  45.37177809  63.65503235]
443 382
[160.9905954  176.10446162 193.40676333]


Detecting objects from videos

In [8]:
video_path = r"car_image.gif"
cap = cv.VideoCapture(video_path)

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
   
size = (frame_width, frame_height)

result = cv.VideoWriter('Output.mp4', 
                         cv.VideoWriter_fourcc(*'FLAC'),
                         10, size)

while True:
    
    ret ,img = cap.read()
    imH ,imW ,ImC = img.shape
    
    frame  = cv.cvtColor(img,cv.COLOR_BGR2RGB)
    tensorimage = tf.convert_to_tensor(frame.copy(),dtype=tf.uint8)
    tensorimage = tensorimage[tf.newaxis,...]
    
    video_detection = model(tensorimage)

    # print(video_detection)

    bbox = video_detection['detection_boxes'][0].numpy()
    scores = video_detection['detection_scores'][0].numpy()
    classnames = video_detection['detection_classes'][0].numpy().astype(np.int32)

    bboxindex = tf.image.non_max_suppression(bbox,scores,max_output_size=50,iou_threshold=0.5,score_threshold=0.5)

    for i in bboxindex:
        box = tuple(bbox[i].tolist())
        ymin ,xmin,ymax,xmax = box
        ymin,xmin,ymax,xmax = (ymin*imH,xmin*imW,ymax*imH,xmax*imW)
        ymin = int(ymin)
        ymax = int(ymax)
        xmin = int(xmin)
        xmax = int(xmax)
        classname = classnames[i]
        classes = names[classname-1]


        cv.rectangle(img,(xmin,ymin),(xmax,ymax),(255,0,0),thickness=2)
        cv.putText(img,classes,(xmin,ymin),cv.FONT_HERSHEY_COMPLEX,1,(255,255,67),1)

    result.write(img)
    cv.imshow('frame',img)
    if cv.waitKey(1) & 0xFF == ord('e'):
        break
cap.release()
cv.destroyAllWindows()

Video(value=b'\x00\x00\x00\x1cftypisom\x00\x00\x02\x00isomiso2mp41\x00\x00\x00\x08free\x00\x08\xc0\x8fmdat\x82…